# Checker Package

**Checker Package: Goal and Purpose**  

The *Checker* package is designed to streamline the creation and evaluation of exercises in Jupyter notebooks. Teachers can define exercises using a dictionary in a notebook cell, ensuring a simple and intuitive setup. Once the *Checker* package is imported, it generates all necessary checking functions automatically, enabling seamless integration with the notebook. This approach minimizes the need for repetitive code, allowing educators to focus on content creation and providing students with an efficient and interactive learning experience.

First step is creating a decorator function that will create the check answer botton and will allow the student to check their answer with the correct solution.

In [14]:
from IPython.display import display
import ipywidgets as widgets

def check(f):
	def wrapper(*args, **kwargs):
		output = widgets.Output()
		button = widgets.Button(description="Check Answer(s)")
		@output.capture(clear_output=True,wait=True)
		def _inner_check(button):
			try:	
				f(*args, **kwargs)
			except:
				print("Something went wrong, have you filled all the functions and run the cells?")
		button.on_click(_inner_check)
		display(button, output)
	return wrapper

We will now create several example with simple fnctions such as multiplication and addition to verify whether the code works.

## Multiplication example

In [15]:
#Checker function for the exercise 
from math import isclose
import numpy as np

@check
def check_example(glob, dict):
    check_float = lambda a, b: isclose(a, b, rel_tol=0, abs_tol=dict["tolerance"])
    if dict["exercise"] == "checking values":

        # Initialize result as an empty list
        result = []
        for i in range(len(dict["values"])):
            # Access global variables dynamically
            result.append(glob[dict["variables"][i]])
            
            # Check if the values match within tolerance
            if check_float(result[i], dict["values"][i]):
                print(f"You got the parameter '{dict['variables'][i]}' right, well done! (checked with tolerance {dict['tolerance']})")
            else:
                print(f"The parameter '{dict['variables'][i]}' is incorrect. (checked with tolerance {dict['tolerance']})")
                print("          Other parts won't be graded until these are fixed.")
                return
            
    if dict["exercise"] == "checking function":
        function = glob[dict["function"]]
	
        test_inputs = dict["inputs"]
        failed = []

        for x, out in test_inputs:
            result = function(x)
            if not check_float(result, out):
                failed.append((x, out, result))

        if len(failed) == 0:
            print(f"Well done, your function is correct! (checked with tolerance {dict['tolerance']})")	
        else:
            print(f"Your function failed some tests. Keep in mind the tolerance is {dict['tolerance']}")
            print("    --> Failed inputs:")
            for case in failed:
                print(f"{case[0]} gave {case[2]}, expected {case[1]}")
            print(f"      Other parts won't be graded until these are fixed.")
            return

	

In [16]:
# Creating a dictionary where the type of exercise and the solution will be stored
my_dict = {
    "exercise": "checking values",
    "tolerance": 0.01,
    "variables": ['multiplication', 'addition'],
    "values": [13.889, 7.53],
}

In [17]:
# exercise script for checking values
# input two values to be multiplied 
x1 = 3.23
x2 = 4.3

# student will insert the result here
multiplication = 13.89
addition = 7.53


In [18]:
check_example(globals(), my_dict)

Button(description='Check Answer(s)', style=ButtonStyle())

Output()

## Function Example 

In [19]:
my_dict2 = {
    "exercise": "checking function",
    "tolerance": 0.01,
    "function": 'find_x_with_probability_p',
    "inputs": [(1 - 0.001, 15.891029744351862), (1 - 0.2, 2.0838374640878863), (1 - 0.9, -3.875794191615308)],
}


#!!!! The dictionaries must be recalled with the above categories name otherwise function won't work 

In [20]:
# exercise script for checking function

def find_x_with_probability_p(p): 
    """ Compute point in the gumbel distribution for which the CDF is p
        Use the variables mu and beta defined above!
        Hint: they have been defined globally, so you don't need to 
                include them as arguments.
    """
    mu = -1.7461
    beta = 2.55342
    x = mu - beta * np.log(-np.log(p))

    return x


In [21]:
check_example(globals(), my_dict2)

Button(description='Check Answer(s)', style=ButtonStyle())

Output()